In [16]:
from dotenv import load_dotenv
from agents import Agent, WebSearchTool, trace, Runner, gen_trace_id, function_tool
from agents.model_settings import ModelSettings
from pydantic import BaseModel, Field
import asyncio
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, From, content
from typing import Dict
from IPython.display  import display, Markdown

In [2]:
load_dotenv (override=True)

True

## OpenAI Hosted Tools

OpenAI Agents SDK includes the following hosted tools:

The `WebSearchTool` lets an agent search the web.  
The `FileSearchTool` allows retrieving information from your OpenAI Vector Stores.  
The `ComputerTool` allows automating computer use tasks like taking screenshots and clicking.


Costs are here: https://platform.openai.com/docs/pricing#web-search

In [4]:
INSTRUCTIONS = "You are a research assistant. Given a search term, you search the web for that term and \
produce a concise summary of the results. The summary must be 2-3 paragraphs and less than 300 \
words. Capture the main points. Write succintly, no need to have complete sentences or good \
grammar. This will be consumed by someone synthesizing a report, so it's vital you capture the \
essence and ignore any fluff. Do not include any additional commentary other than the summary itself"

search_agent = Agent(
    name = "search agent",
    instructions=INSTRUCTIONS,
    model='gpt-4o-mini',
    tools = [WebSearchTool(search_context_size="low")],
    model_settings=ModelSettings(tool_choice="required")

)

In [5]:
OPEN_AI_KEY = os.getenv("OPENAI_API_KEY")

In [6]:
if OPEN_AI_KEY:
    print(f"Open AI key is found and starts with {OPEN_AI_KEY[:8]}")
else:
    print(f"Open AI key is not found. Check env and docs")


Open AI key is found and starts with sk-proj-


In [ ]:
message = "Latest AI Agent frameworks in 2025"

with trace("Latest Agentic AI framweork"):
    result = await Runner.run(search_agent, message)
    
    
display(Markdown(result.final_output))



In 2025, several AI agent frameworks have emerged, enhancing the development and deployment of intelligent agents across various applications. Dalet introduced "Dalia," an agentic AI solution integrated throughout its media ecosystem, offering a unified, natural-language interface for tasks like asset management and live ingests. ([tvtechnology.com](https://www.tvtechnology.com/news/dalet-introduces-dalia-agentic-ai-across-entire-ecosystem?utm_source=openai)) Cisco unveiled "Connected Intelligence," a suite of AI agents within its Webex platform, including task agents, notetakers, and meeting schedulers, aiming to boost productivity in collaboration workflows. ([techradar.com](https://www.techradar.com/pro/cisco-goes-all-in-on-agents-and-it-could-mean-big-changes-in-your-workplace?utm_source=openai)) OutSystems launched "Agent Workbench," a low-code platform enabling enterprises to implement agentic AI for streamlining operations and modernizing legacy systems. ([techradar.com](https://www.techradar.com/pro/outsystems-agent-workbench-reaches-general-availability-helping-enterprises-streamline-operations-through-agentic-ai?utm_source=openai))

Academically, frameworks like "Agent Lightning" and "Cognitive Kernel-Pro" have been developed. Agent Lightning is a flexible framework that decouples agent execution from training, facilitating reinforcement learning-based training of large language models for any AI agent. ([arxiv.org](https://arxiv.org/abs/2508.03680?utm_source=openai)) Cognitive Kernel-Pro is an open-source, multi-module agent framework designed to democratize the development and evaluation of advanced AI agents, achieving state-of-the-art results among open-source agents. ([arxiv.org](https://arxiv.org/abs/2508.00414?utm_source=openai)) Additionally, the Model Context Protocol (MCP), introduced by Anthropic in November 2024, has been adopted by major AI providers, including OpenAI and Google DeepMind, to standardize AI system integration and data sharing. ([en.wikipedia.org](https://en.wikipedia.org/wiki/Model_Context_Protocol?utm_source=openai)) 

#### PLANNER AGENTS

Planner Agents - It will receive a query and come up with a handful of searches that it should run. Limiting the search to three for this purpose/project due to costing

In [10]:
Number_of_Searches = 3
#This is the number of searches it should run

INSTRUCTIONS = f"You are a helpful research assistant. Given a query , come up with a set of web services \
    to perform to best answer the query. Output {Number_of_Searches} terms to query for."

class WebSearchItem(BaseModel):
    reason:str
    "Your reasoning for why this search is important to the query"


    query:str
    "The search term to use for the web search"


class WebSearchPlan(BaseModel):

    searches: list[WebSearchItem]
    """A list of web searches to perform to best answer the query."""




planner_agent = Agent(
    name = "PlannerAgent",
    instructions = INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=WebSearchPlan
)

In [12]:
message = "Latest Agentic AI Frameworks in 2025"

with trace("Search"):
    result = await Runner.run(planner_agent,message)

    print(result.final_output)


searches=[WebSearchItem(reason='To gather information about the most recent developments and frameworks in agentic AI expected in 2025.', query='latest agentic AI frameworks 2025'), WebSearchItem(reason='To explore predictions and trends in technology specifically regarding agentic AI in 2025.', query='agentic AI trends 2025'), WebSearchItem(reason='To find scholarly articles, white papers, or technical reports on agentic AI frameworks that will be relevant in 2025.', query='agentic AI research papers 2025')]


In [13]:
@function_tool
def send_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Send out an email with the given subject and HTML body """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("toyeladun@gmail.com") # Change this to your verified email
    to_email = To("toyeladunr@gmail.com") # Change this to your email
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    response = sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

In [14]:
INSTRUCTIONS = """You are able to send a nicely formatted HTML email based on a detailed report.
You will be provided with a detailed report. You should use your tool to send one email, providing the 
report converted into clean, well presented HTML with an appropriate subject line."""

email_agent = Agent(
    name="Email agent",
    instructions=INSTRUCTIONS,
    tools=[send_email],
    model="gpt-4o-mini",
)

In [17]:
INSTRUCTIONS = (
    "You are a senior researcher tasked with writing a cohesive report for a research query. "
    "You will be provided with the original query, and some initial research done by a research assistant.\n"
    "You should first come up with an outline for the report that describes the structure and "
    "flow of the report. Then, generate the report and return that as your final output.\n"
    "The final output should be in markdown format, and it should be lengthy and detailed. Aim "
    "for 5-10 pages of content, at least 1000 words."
)


class ReportData(BaseModel):
    short_summary: str = Field(description="A short 2-3 sentence summary of the findings.")

    markdown_report: str = Field(description="The final report")

    follow_up_questions: list[str] = Field(description="Suggested topics to research further")


writer_agent = Agent(
    name="WriterAgent",
    instructions=INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=ReportData,
)

In [20]:
async def plan_searches(query: str):
    """ Use the planner_agent to plan which searches to run for the query """
    print("Planning searches...")
    result = await Runner.run(planner_agent, f"Query: {query}")
    print(f"Will perform {len(result.final_output.searches)} searches")
    return result.final_output

async def perform_searches(search_plan: WebSearchPlan):
    """ Call search() for each item in the search plan """
    print("Searching...")
    tasks = [asyncio.create_task(search(item)) for item in search_plan.searches]
    results = await asyncio.gather(*tasks)
    print("Finished searching")
    return results

async def search(item: WebSearchItem):
    """ Use the search agent to run a web search for each item in the search plan """
    input = f"Search term: {item.query}\nReason for searching: {item.reason}"
    result = await Runner.run(search_agent, input)
    return result.final_output

In [18]:
async def write_report(query: str, search_results: list[str]):
    """ Use the writer agent to write a report based on the search results"""
    print("Thinking about report...")
    input = f"Original query: {query}\nSummarized search results: {search_results}"
    result = await Runner.run(writer_agent, input)
    print("Finished writing report")
    return result.final_output

async def send_email(report: ReportData):
    """ Use the email agent to send an email with the report """
    print("Writing email...")
    result = await Runner.run(email_agent, report.markdown_report)
    print("Email sent")
    return report

In [ ]:
query ="Latest AI Agent frameworks in 2025"

with trace("Research trace"):
    print("Starting research...")
    search_plan = await plan_searches(query)
    search_results = await perform_searches(search_plan)
    report = await write_report(query, search_results)
    await send_email(report)  
    print("Hooray!")

Starting research...
Planning searches...
Will perform 1 searches
Searching...
Finished searching
Thinking about report...
Finished writing report
Writing email...
